# 🌊 Marine Carbonate System - TA & pCO2

**Interactive Bjerrum Plot** using Total Alkalinity and pCO2 parameters.

**Quick Instructions:**
- Adjust sliders to change TA and pCO2 values
- Observe ocean acidification effects
- Use preset buttons for different scenarios

---
**Author:** Cardoso-Mohedano J.G.  
**Institution:** Instituto de Ciencias del Mar y Limnología, UNAM

In [6]:
# Essential imports only
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output, Button, HTML
from IPython.display import display

In [7]:
# Simplified calculation function
def calc_system(alk, pco2):
    """Calculate carbonate system from TA and pCO2"""
    results = pyco2.sys(par1=alk, par2=pco2, par1_type=1, par2_type=4,
                       salinity=35, temperature=25, pressure=0,
                       opt_pH_scale=1, opt_k_carbonic=10)
    return {
        'pH': float(results['pH_total']),
        'pCO2': float(results['pCO2']),
        'HCO3': float(results['bicarbonate']),
        'CO3': float(results['carbonate']),
        'DIC': float(results['dic']),
        'omega': float(results['saturation_aragonite'])
    }

def plot_system(data):
    """Create simplified 4-panel plot with marine color palette"""
    # Paleta "Colores del Mar"
    azul_profundo = '#023859'    # Fondo o trazos dominantes
    verde_aqua = '#2A9D8F'       # Resaltado o barras positivas
    azul_claro = '#87C4D9'       # Rellenos suaves o líneas auxiliares
    arena_calida = '#E9C46A'     # Resaltado neutro o categorías
    coral_vivo = '#F4A261'       # Alertas o valores negativos
    rojo_atardecer = '#E76F51'   # Advertencias o zonas críticas
    
    plt.close('all')
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    fig.patch.set_facecolor('white')
    
    # Bjerrum plot
    pH_range = np.linspace(5, 10, 30)
    H = 10**(-pH_range)
    K1, K2 = 1e-6, 1e-9  # Simplified constants
    denom = H**2 + K1*H + K1*K2
    
    ax1.plot(pH_range, H**2/denom, color=azul_profundo, linewidth=2.5, label='CO2')
    ax1.plot(pH_range, K1*H/denom, color=verde_aqua, linewidth=2.5, label='HCO3-')
    ax1.plot(pH_range, K1*K2/denom, color=azul_claro, linewidth=2.5, label='CO3-2')
    ax1.axvline(data['pH'], color=coral_vivo, linestyle='--', linewidth=2)
    ax1.set_xlabel('pH'), ax1.set_ylabel('Fraction')
    ax1.set_title('Bjerrum Plot'), ax1.legend(), ax1.grid(True, alpha=0.3)
    
    # Composition pie
    CO2_conc = data['pCO2'] * 0.034
    species = [CO2_conc, data['HCO3'], data['CO3']]
    colors_pie = [azul_profundo, verde_aqua, azul_claro]
    ax2.pie(species, labels=['CO2*', 'HCO3-', 'CO3-2'], autopct='%1.1f%%', colors=colors_pie)
    ax2.set_title(f'Composition at pH {data["pH"]:.2f}')
    
    # Saturation
    omega = data['omega']
    color = verde_aqua if omega >= 1 else rojo_atardecer
    ax3.bar(['Aragonite'], [omega], color=color)
    ax3.axhline(1, color=azul_profundo, linestyle='--')
    ax3.set_ylabel('Omega'), ax3.set_title('Saturation State')
    
    # Results
    ax4.axis('off')
    results_text = f"""pH = {data['pH']:.2f}
pCO2 = {data['pCO2']:.0f} μatm
HCO3- = {data['HCO3']:.0f} μmol/kg
CO3-2 = {data['CO3']:.0f} μmol/kg
DIC = {data['DIC']:.0f} μmol/kg
Ω_arag = {omega:.2f}

Conditions:
S = 35 PSU, T = 25°C"""
    ax4.text(0.1, 0.9, results_text, transform=ax4.transAxes, 
             fontsize=10, verticalalignment='top', fontfamily='monospace', color=azul_profundo)
    
    plt.tight_layout()
    plt.show()
    return fig

In [8]:
# Create interface
alk_slider = FloatSlider(value=2300, min=1800, max=3000, step=50,
                        description="Alkalinity:", style={'description_width': 'initial'})
pco2_slider = FloatSlider(value=400, min=200, max=1200, step=25,
                         description="pCO2 (μatm):", style={'description_width': 'initial'})

def update_plot(alk, pco2):
    """Update plot when sliders change"""
    data = calc_system(alk, pco2)
    plot_system(data)

# Preset buttons
def reset_vals(b): alk_slider.value, pco2_slider.value = 2300, 400
def preindustrial(b): alk_slider.value, pco2_slider.value = 2300, 280
def current(b): alk_slider.value, pco2_slider.value = 2300, 420
def future(b): alk_slider.value, pco2_slider.value = 2300, 800

reset_btn = Button(description="Reset", button_style='info')
pre_btn = Button(description="Pre-industrial", button_style='success')
current_btn = Button(description="Current", button_style='warning')
future_btn = Button(description="Future", button_style='danger')

reset_btn.on_click(reset_vals)
pre_btn.on_click(preindustrial)
current_btn.on_click(current)
future_btn.on_click(future)

# Display interface
display(VBox([
    HTML("<h3>🌊 Ocean Acidification Explorer</h3>"),
    alk_slider, pco2_slider,
    VBox([pre_btn, current_btn, future_btn, reset_btn]),
    interactive_output(update_plot, {'alk': alk_slider, 'pco2': pco2_slider})
]))

print("✅ Fast ocean acidification interface ready!")
print("🎯 Optimized for quick Binder loading")

print("Author:Cardoso-Mohedano J.G. UNAM")

✅ Fast ocean acidification interface ready!
🎯 Optimized for quick Binder loading
Author:Cardoso-Mohedano J.G. UNAM
